In [ ]:
import xlrd
import glob
import csv
import re
import sys
import time
#from multiprocessing import Pool
from multiprocessing.dummy import Pool

In [ ]:
header = None
def process_xls(filename):
    global header
    book = xlrd.open_workbook(filename, on_demand=True)
    #f = open(filename[:-4] + ".csv", 'wb')
    
    f = open("combined.csv", 'a')
    output_writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)

    print ("Opened ", filename, " with ", book.nsheets, " sheets.")
    #print book.sheet_names()
    total_rows = 0
    processed_rows = 0
    for i in range(book.nsheets):
        sheet = book.sheet_by_index(i)
        print ("sheet:", i, sheet.name)
        total_rows += sheet.nrows
        
        offset = 0
        if header is None:
            header = sheet.row_values(0)            
        elif sheet.row_values(0) == header:
            offset = 1
            
        for row in range(offset, sheet.nrows):
            row_values = sheet.row_values(row)
            row_types = sheet.row_types(row)
            for idx, typ in enumerate(row_types):
                if typ == xlrd.XL_CELL_NUMBER:
                    t = float(row_values[idx])
                    if t.is_integer():
                        row_values[idx] = int(t)
                    else: 
                        row_values[idx] = t
            output_writer.writerow(row_values)
            processed_rows += 1
        book.unload_sheet(i)
    book.release_resources()
    f.close()
    print ("Wrote ", processed_rows, "of ", total_rows, " total rows")
    return (processed_rows, total_rows)

In [ ]:
start = time.time() 
xls_names = glob.glob("*.xls")   

#header_values = get_header(file_names[0]) # write header
#print "Header values: ", header_values
#output_writer.writerow(header_values) # writer.send(header_values)

"""
print "Starting pools."   
pool = Pool(4) # Sets the pool size to 4
pool.imap_unordered(process_xls, xls_names)
pool.close() # no more work
pool.join() # wait for completion
"""
results = []
for file in xls_names:
    result = process_xls(file)
    results.append(result)

end = time.time()
elapsed_seconds = float("%.2f" % (end - start))
print ("Execution time (seconds):", elapsed_seconds)

processed, total = 0, 0
for (p, t) in results:
    processed += p
    total += t
print ("Processed %s out of total %s" % (processed, total))

#csv_names = glob.glob("tmp\*.csv")
#for csv in csv_names:
#    os.remove(csv)
#os.rmdir("tmp")

In [ ]:
processed, total = 0, 0
for (p, t) in results:
    processed += p
    total += t
print "Processed %s out of total %s" % (processed, total)